In [1]:
import os
import pandas as pd

I have a lot of data, which is partially cleaned. My previous workbook was getting pretty long, so I decied to start with a fresh one to finish cleaning the data and then I will merge it with the relavant season stats and we'll be able to start creating some visualizations and frame the way the models will work so that its is as accurate as possible in a real-world environment.

In [4]:
GAMES_PATH = "data/games/"
GAMES_DATA_PATH = "data/GAMES_CSV/"

TEAMS = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KAN', 'LAA', 'LAD', 'MIA', 'MIL', 
                 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF', 'SEA', 'STL', 'TB','TEX', 'TOR', 'WSN']
YEARS = list(range(2012, 2023))

for team in TEAMS:
    for year in YEARS:
        
        file_string = f"{GAMES_DATA_PATH}{team}_{year}_games.csv"

        try:
            df = pd.read_csv(file_string) 

        except:
            print(f"File missing: {file_string}")

print("Done")


File missing: data/GAMES_CSV/ARI_2016_games.csv
File missing: data/GAMES_CSV/KAN_2016_games.csv
File missing: data/GAMES_CSV/KAN_2017_games.csv
File missing: data/GAMES_CSV/KAN_2018_games.csv
File missing: data/GAMES_CSV/KAN_2019_games.csv
File missing: data/GAMES_CSV/KAN_2020_games.csv
File missing: data/GAMES_CSV/KAN_2021_games.csv
File missing: data/GAMES_CSV/KAN_2022_games.csv
File missing: data/GAMES_CSV/NYM_2018_games.csv
File missing: data/GAMES_CSV/TB_2019_games.csv
Done


I want to verify the counts of the files for each team to make sure it aligns with the results from above. So lets write that function

In [10]:
for team in TEAMS:
    season_counter = 0
    seasons = []
    
    for year in YEARS:
        
        file_string = f"{GAMES_DATA_PATH}{team}_{year}_games.csv"

        try:
            df = pd.read_csv(file_string) 
            season_counter+=1

        except:
            seasons.append(year)

    if season_counter < 11:
        #print(f"{team} Season Files Count: {season_counter}")
        print(f"{team} Missing Seasons: {seasons}\n")
    

print("Done")

ARI Missing Seasons: [2016]

KAN Missing Seasons: [2016, 2017, 2018, 2019, 2020, 2021, 2022]

NYM Missing Seasons: [2018]

TB Missing Seasons: [2019]

Done


Now that we know which seasons are missing from our csv data set, we can go back to our web scrapping script and get these specific seasons. We'll then run it through the cleaning script from before and we will have all the seasons from 2012 - 2022 in a format that can be used to look at for answers and train models.

In [15]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [18]:
#Function to scrape html from a webpage

async def get_html(url, selector, sleep=3, retries=10):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            print(f"Scraping page. Attempt: {i}")
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [21]:
#Scrape content from each mlb game of season. Start with 2012 b/c miami marlines began then
Team_LIST = ['ARI', 'KCR', 'NYM', 'TBR']
GAMES_SAVE_PATH = "data/games/"

# ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KAN', 'LAA', 'LAD', 'MIA', 'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF', 'SEA', 'STL', 'TB',
#Season = 2012


async def scrape_games(Team, Season):
    try:
        URL_2 = f"https://www.baseball-reference.com/teams/{Team}/{Season}-schedule-scores.shtml"
    
        save_name = f"{Team}_{Season}"
        save_path = os.path.join(GAMES_SAVE_PATH, save_name)
    
        html = await get_html(URL_2, "#content")
        if (html):
            with open(save_path, "w+") as f:
                f.write(html)
                
    except Exception as e:
        print(e)
    

In [22]:
seasons = list(range(2016, 2023))

for season in seasons:
    await scrape_games('KCR', season)


print("Done")

Scraping page. Attempt: 1
Timeout error on https://www.baseball-reference.com/teams/KCR/2016-schedule-scores.shtml
Scraping page. Attempt: 2
Timeout error on https://www.baseball-reference.com/teams/KCR/2016-schedule-scores.shtml
Scraping page. Attempt: 3
Timeout error on https://www.baseball-reference.com/teams/KCR/2016-schedule-scores.shtml
Scraping page. Attempt: 4
2016 Kansas City Royals Schedule | Baseball-Reference.com
Scraping page. Attempt: 1
2017 Kansas City Royals Schedule | Baseball-Reference.com
Scraping page. Attempt: 1
Timeout error on https://www.baseball-reference.com/teams/KCR/2018-schedule-scores.shtml
Scraping page. Attempt: 2
Timeout error on https://www.baseball-reference.com/teams/KCR/2018-schedule-scores.shtml
Scraping page. Attempt: 3
2018 Kansas City Royals Schedule | Baseball-Reference.com
Scraping page. Attempt: 1
Timeout error on https://www.baseball-reference.com/teams/KCR/2019-schedule-scores.shtml
Scraping page. Attempt: 2
2019 Kansas City Royals Schedule

In [23]:
seasons = list(range(2012, 2016))

for season in seasons:
    await scrape_games('KCR', season)


print("Done")

Scraping page. Attempt: 1
Timeout error on https://www.baseball-reference.com/teams/KCR/2012-schedule-scores.shtml
Scraping page. Attempt: 2
Timeout error on https://www.baseball-reference.com/teams/KCR/2012-schedule-scores.shtml
Scraping page. Attempt: 3
2012 Kansas City Royals Schedule | Baseball-Reference.com
Scraping page. Attempt: 1
2013 Kansas City Royals Schedule | Baseball-Reference.com
Scraping page. Attempt: 1
2014 Kansas City Royals Schedule | Baseball-Reference.com
Scraping page. Attempt: 1
Timeout error on https://www.baseball-reference.com/teams/KCR/2015-schedule-scores.shtml
Scraping page. Attempt: 2
Timeout error on https://www.baseball-reference.com/teams/KCR/2015-schedule-scores.shtml
Scraping page. Attempt: 3
Timeout error on https://www.baseball-reference.com/teams/KCR/2015-schedule-scores.shtml
Scraping page. Attempt: 4
Timeout error on https://www.baseball-reference.com/teams/KCR/2015-schedule-scores.shtml
Scraping page. Attempt: 5
Timeout error on https://www.bas

So now let's check to see if we have all the files we need for Kansas City, with the new key name: KCR instead of KAS

In [30]:
GAMES_PATH = "data/games/"
GAMES_DATA_PATH = "data/games/"

TEAMS = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KCR', 'LAA', 'LAD', 'MIA', 'MIL', 
                 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF', 'SEA', 'STL', 'TB','TEX', 'TOR', 'WSN']
YEARS = list(range(2012, 2023))

for team in TEAMS:
    for year in YEARS:
        
        file_string = f"{GAMES_DATA_PATH}{team}_{year}"
       
        try:
            f = open(file_string, 'r')
            df = pd.read_html(f) 

        except:
            print(f"File missing: {file_string}")

print("Done")

File missing: data/games/ARI_2016
File missing: data/games/NYM_2018
File missing: data/games/TB_2019
Done


So we can see we have 3 more files to grab anad then we will need to convert all our files to a csv from its html file

In [31]:
# Grab Arizona 2016
await scrape_games('ARI', 2016)

Scraping page. Attempt: 1
2016 Arizona Diamondbacks Schedule | Baseball-Reference.com


In [32]:
# Grab Mets 2018
await scrape_games('NYM', 2018)

Scraping page. Attempt: 1
Timeout error on https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml
Scraping page. Attempt: 2
Timeout error on https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml
Scraping page. Attempt: 3
Timeout error on https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml
Scraping page. Attempt: 4
Timeout error on https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml
Scraping page. Attempt: 5
Timeout error on https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml
Scraping page. Attempt: 6
Timeout error on https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml
Scraping page. Attempt: 7
Timeout error on https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml
Scraping page. Attempt: 8
Timeout error on https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml
Scraping page. Attempt: 9
2018 New York Mets Schedule | Baseball-Reference.com


In [33]:
# Grab Tampa Bay 2019
await scrape_games('TB', 2019)

Scraping page. Attempt: 1
Timeout error on https://www.baseball-reference.com/teams/TB/2019-schedule-scores.shtml
Scraping page. Attempt: 2
2019 Tampa Bay Rays Schedule | Baseball-Reference.com


In [34]:
GAMES_PATH = "data/games/"
GAMES_SAVE_PATH = "data/GAMES_CSV/"


#Now lets loop through each file and convert it
#year = 2012


def Extract_Games_and_Save(Team, Year):
    FILE_NAME = f"{GAMES_PATH}{Team}_{Year}"

    f = open(FILE_NAME, 'r')
    schedule_df = pd.read_html(f)
  
    if (schedule_df):
        games_table = schedule_df[4] #Games
      
        #save path Write to csv
        table_path_1 = f"{GAMES_SAVE_PATH}{Team}_{Year}_games.csv"
        games_table.to_csv(table_path_1)
        

In [35]:
seasons = list(range(2012, 2023))

for season in seasons:
    Extract_Games_and_Save('KCR', season)

In [36]:
#Lets do the same for the other three files

Extract_Games_and_Save('ARI', 2016)
Extract_Games_and_Save('NYM', 2018)
Extract_Games_and_Save('TB', 2019)

In [38]:
#And now lets check our counts for each team
GAMES_DATA_PATH = "data/GAMES_CSV/"


for team in TEAMS:
    season_counter = 0
    
    for year in YEARS:
        
        file_string = f"{GAMES_DATA_PATH}{team}_{year}_games.csv"

        try:
            
            df = pd.read_csv(file_string) 
            season_counter+=1

        except:
            None

   
    print(f"{team} Season Files Count: {season_counter}")
        

print("Done")

ARI Season Files Count: 11
ATL Season Files Count: 11
BAL Season Files Count: 11
BOS Season Files Count: 11
CHC Season Files Count: 11
CHW Season Files Count: 11
CIN Season Files Count: 11
CLE Season Files Count: 11
COL Season Files Count: 11
DET Season Files Count: 11
HOU Season Files Count: 11
KCR Season Files Count: 11
LAA Season Files Count: 11
LAD Season Files Count: 11
MIA Season Files Count: 11
MIL Season Files Count: 11
MIN Season Files Count: 11
NYM Season Files Count: 11
NYY Season Files Count: 11
OAK Season Files Count: 11
PHI Season Files Count: 11
PIT Season Files Count: 11
SD Season Files Count: 11
SF Season Files Count: 11
SEA Season Files Count: 11
STL Season Files Count: 11
TB Season Files Count: 11
TEX Season Files Count: 11
TOR Season Files Count: 11
WSN Season Files Count: 11
Done


Wonderful, and now that we have a full 11 seasons worth of data for each team in a csv file, we can redo our cleaning scripts to get processed games for each team

In [39]:
def convert_win_streak(streak):
    if streak[0] == '+':
        return len(streak)
    else:
        return (-len(streak))

def convert_GB(gb):
    if gb[0] == 'u':
        gb = gb[2:]
        return float(gb)
    elif gb[0] == 'T':
        return 0
    else:
        return -float(gb)


def convert_outcome(outcome):
    if (outcome[0] == 'W'):
        return 1
    else:
        return 0

def convert_walkoff(outcome):
    if (outcome == 'W') or (outcome == 'L'):
        return 0
    else:
        if (outcome[0] == 'W'):
            return 1
        else:
            return -1


def convert_time_to_min(t):
    hour = t[0]
    min = t[2:]

    return ((int(hour)*60) + int(min))


def convert_day(x):
    if x == 'D':
        return 1
    else:
        return 0

def convert_night(x):
    if x == 'N':
        return 1
    else:
        return 0


def convert_date_to_datetime(day):
    if (day[-1] != ')'):
        return day[-6:]
    else:
        return day[-10:-4]


def convert_total_wins(record):
    return int(record.split('-')[0])

def convert_total_losses(record):
    return int(record.split('-')[1])



def clean_df(df):
    df['GameID'] = df['Gm#']
    df['Team'] = df['Tm']
    df['Outcome'] = df['W/L']
    df['W-L_record'] = df['W-L']
    df['Extra_Innings'] = df['Inn']
    df['Day-or-Night_g'] = df['D/N']
    df['champ_lev_index'] = df['cLI']
    df['Streaks'] = df['Streak'].apply(lambda x: convert_win_streak(x))
    
    del df['Orig. Scheduled']
    del df['Gm#']
    del df['Tm']
    del df['W/L']
    del df['W-L']
    del df['Inn']
    del df['D/N']
    del df['cLI']
    del df['Streak']
    del df['Unnamed: 0']
    del df['Unnamed: 2']
    del df['Unnamed: 4']
    del df['Win']
    del df['Loss']
    del df['Save']

    df = df[['Date', 'GameID', 'Team', 'Opp', 'Outcome', 'R', 'RA', 'W-L_record', 'Streaks', 'Rank', 'GB', 'Time', 'Attendance',
               'Extra_Innings', 'Day-or-Night_g', 'champ_lev_index']]

    df = df[df.GameID != 'Gm#']

    return df

In [40]:
def Process_Team_Schedules(Team, Year):
    GAMES_DATA_PATH = "data/GAMES_CSV/"

    file_string = f"{GAMES_DATA_PATH}{Team}_{Year}_games.csv"

    
    df = pd.read_csv(file_string) 
    
    df = clean_df(df)
    
    df['GB'] = df['GB'].apply(lambda x: convert_GB(x))
    
    df['Walkoff'] = df['Outcome']
    df['Outcome'] = df['Outcome'].apply(lambda x: convert_outcome(x))
    df['Walkoff'] = df['Walkoff'].apply(lambda x: convert_walkoff(x))
    df["Total_Innings"] = df['Extra_Innings'].fillna('9')
    del df['Extra_Innings']
    
    df['Time_Min'] = df['Time'].apply(lambda x: convert_time_to_min(x))
    del df['Time']
    
    df['is_day_game'] = df['Day-or-Night_g'].apply(lambda x: convert_day(x))
    df['is_night_game'] = df['Day-or-Night_g'].apply(lambda x: convert_night(x))
    del df['Day-or-Night_g']
    
    df['new_date'] = df['Date'].apply(lambda x: convert_date_to_datetime(x))
    
    test_dates = df['new_date']
    test_dates = pd.to_datetime(test_dates + ' ' + str(Year))
    
    df['y-m-d'] = test_dates
    del df['new_date']
    
    df['total_wins'] = df['W-L_record'].apply(lambda x: convert_total_wins(x))
    
    df['total_losses'] = df['W-L_record'].apply(lambda x: convert_total_losses(x))
    del df['W-L_record']
    
    del df['Date']
    
    df = df[['y-m-d', 'GameID',  'Time_Min', 'Team', 'Opp', 'R', 'RA', 'total_wins', 'total_losses',
                       'Total_Innings', 'Walkoff', 'Streaks', 'Rank', 'GB', 'Attendance', 'champ_lev_index', 'is_day_game', 'is_night_game', 'Outcome']]
    
    save_path = f"data/PROCESSED_GAMES/{Team}_{Year}.csv"
    df.to_csv(save_path)

In [41]:
def Process_Teams(YEAR):
    Team_LIST = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KCR', 'LAA', 'LAD', 'MIA', 'MIL', 
                 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF', 'SEA', 'STL', 'TB','TEX', 'TOR', 'WSN']
    
    for Team in Team_LIST:
        Process_Team_Schedules(Team, YEAR)

In [42]:
#Now loop through all year files
YEARS = list(range(2012, 2023))

for year in YEARS:
    Process_Teams(year)

Lets now check to make sure that we have a full 11 seasons worth of processed data for each team

In [43]:
#And now lets check our counts for each team
GAMES_DATA_PATH = "data/PROCESSED_GAMES/"


for team in TEAMS:
    season_counter = 0
    
    for year in YEARS:
        file_string = f"{GAMES_DATA_PATH}{team}_{year}.csv"

        try:
            df = pd.read_csv(file_string) 
            season_counter+=1
        except:
            None

    print(f"{team} Season Files Count: {season_counter}")
        

print("Done")

ARI Season Files Count: 11
ATL Season Files Count: 11
BAL Season Files Count: 11
BOS Season Files Count: 11
CHC Season Files Count: 11
CHW Season Files Count: 11
CIN Season Files Count: 11
CLE Season Files Count: 11
COL Season Files Count: 11
DET Season Files Count: 11
HOU Season Files Count: 11
KCR Season Files Count: 11
LAA Season Files Count: 11
LAD Season Files Count: 11
MIA Season Files Count: 11
MIL Season Files Count: 11
MIN Season Files Count: 11
NYM Season Files Count: 11
NYY Season Files Count: 11
OAK Season Files Count: 11
PHI Season Files Count: 11
PIT Season Files Count: 11
SD Season Files Count: 11
SF Season Files Count: 11
SEA Season Files Count: 11
STL Season Files Count: 11
TB Season Files Count: 11
TEX Season Files Count: 11
TOR Season Files Count: 11
WSN Season Files Count: 11
Done


In [44]:
GAMES_DATA_PATH = "data/PROCESSED_GAMES/"
Team = 'ARI'
Years = list(range(2012, 2023))

concated_df = pd.DataFrame()


for year in Years:
    file_string_1 = f"{GAMES_DATA_PATH}{Team}_{year}.csv"
    df1 = pd.read_csv(file_string_1) 
    del df1['Unnamed: 0']
    concated_df = pd.concat([concated_df, df1], axis=0)


In [45]:
concated_df

,y-m-d,GameID,Time_Min,Team,Opp,R,RA,total_wins,total_losses,Total_Innings,Walkoff,Streaks,Rank,GB,Attendance,champ_lev_index,is_day_game,is_night_game,Outcome
0,2012-04-06,1,173,ARI,SFG,5,4,1,0,9,0,1,2,-0.5,49130.0,1.04,1,0,1
1,2012-04-07,2,156,ARI,SFG,5,4,2,0,9,0,2,2,-0.5,34789.0,1.11,1,0,1
2,2012-04-08,3,188,ARI,SFG,7,6,3,0,9,0,3,1,0.5,24193.0,1.16,1,0,1
3,2012-04-10,4,208,ARI,SDP,4,2,4,0,11,0,4,1,0.5,18652.0,1.20,0,1,1
4,2012-04-11,5,178,ARI,SDP,1,2,4,1,9,0,-1,2,-0.5,16091.0,1.19,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,2022-10-01,158,190,ARI,SFG,8,4,73,85,9,0,1,4,-37.0,30630.0,0.00,1,0,1
158,2022-10-02,159,216,ARI,SFG,3,4,73,86,10,-1,-1,4,-37.0,34824.0,0.00,1,0,0
159,2022-10-03,160,199,ARI,MIL,5,6,73,87,10,-1,-2,4,-37.0,18612.0,0.00,0,1,0
160,2022-10-04,161,182,ARI,MIL,0,3,73,88,9,0,-3,4,-37.0,19611.0,0.00,0,1,0


In [54]:
GAMES_DATA_PATH = "data/PROCESSED_GAMES/"

def Merge_Team_Seasons(Team):
    concated_df = pd.DataFrame()
    
    Years = list(range(2012, 2023))
    
    for year in Years:
        file_string_1 = f"{GAMES_DATA_PATH}{Team}_{year}.csv"
        df1 = pd.read_csv(file_string_1) 
        del df1['Unnamed: 0']
        concated_df = pd.concat([concated_df, df1], axis=0)
        save_string_1 = f"data/TEAM_SEASONS/{Team}_2012-2022.csv"
        concated_df.to_csv(save_string_1, index=False)

    print(f"Wrote: {Team}")

In [52]:
Merge_Team_Seasons('ARI')

In [55]:
#Now write loop to do this for all teams

TEAMS = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KCR', 'LAA', 'LAD', 'MIA', 'MIL', 
             'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF', 'SEA', 'STL', 'TB','TEX', 'TOR', 'WSN']

for team in TEAMS:
    Merge_Team_Seasons(team)

Wrote: ARI
Wrote: ATL
Wrote: BAL
Wrote: BOS
Wrote: CHC
Wrote: CHW
Wrote: CIN
Wrote: CLE
Wrote: COL
Wrote: DET
Wrote: HOU
Wrote: KCR
Wrote: LAA
Wrote: LAD
Wrote: MIA
Wrote: MIL
Wrote: MIN
Wrote: NYM
Wrote: NYY
Wrote: OAK
Wrote: PHI
Wrote: PIT
Wrote: SD
Wrote: SF
Wrote: SEA
Wrote: STL
Wrote: TB
Wrote: TEX
Wrote: TOR
Wrote: WSN


Alright, so now we have 30 files each with a team's season data in it. I want to make one more file that is a merging of all the season data called "TOTAL_TEAM_SEASONS_2012-2022.csv"

In [57]:
GAMES_DATA_PATH = "data/TEAM_SEASONS/"

TEAMS = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KCR', 'LAA', 'LAD', 'MIA', 'MIL', 
             'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF', 'SEA', 'STL', 'TB','TEX', 'TOR', 'WSN']


concated_df = pd.DataFrame()
    
for team in TEAMS:
    file_string_1 = f"{GAMES_DATA_PATH}{team}_2012-2022.csv"
    df1 = pd.read_csv(file_string_1) 
    
    concated_df = pd.concat([concated_df, df1], axis=0)
    
    save_string_1 = f"data/TEAM_SEASONS/TOTAL_TEAM_SEASONS_2012-2022.csv"
    concated_df.to_csv(save_string_1, index=False)

print("Done")

Done


In [58]:
#lets look at our total seasons df

file_string_1 = "data/TEAM_SEASONS/TOTAL_TEAM_SEASONS_2012-2022.csv"
df1 = pd.read_csv(file_string_1) 
df1

,y-m-d,GameID,Time_Min,Team,Opp,R,RA,total_wins,total_losses,Total_Innings,Walkoff,Streaks,Rank,GB,Attendance,champ_lev_index,is_day_game,is_night_game,Outcome
0,2012-04-06,1,173,ARI,SFG,5,4,1,0,9,0,1,2,-0.5,49130.0,1.04,1,0,1
1,2012-04-07,2,156,ARI,SFG,5,4,2,0,9,0,2,2,-0.5,34789.0,1.11,1,0,1
2,2012-04-08,3,188,ARI,SFG,7,6,3,0,9,0,3,1,0.5,24193.0,1.16,1,0,1
3,2012-04-10,4,208,ARI,SDP,4,2,4,0,11,0,4,1,0.5,18652.0,1.20,0,1,1
4,2012-04-11,5,178,ARI,SDP,1,2,4,1,9,0,-1,2,-0.5,16091.0,1.19,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50385,2022-10-01,158,169,WSN,PHI,2,8,55,103,9,0,-1,5,-44.0,36841.0,0.00,0,1,0
50386,2022-10-02,159,140,WSN,PHI,1,8,55,104,6,0,-2,5,-45.0,32779.0,0.00,1,0,0
50387,2022-10-04,160,192,WSN,NYM,2,4,55,105,9,0,-3,5,-46.0,NaN,0.00,0,1,0
50388,2022-10-04,161,189,WSN,NYM,0,8,55,106,9,0,-4,5,-46.0,23649.0,0.00,0,1,0
